# ETL Y EDA INVENTARIO FINAL 2016

In [19]:
#LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
#Cargamos el archivo csv y nombramos dataset
inv_final = pd.read_csv("EndInvFINAL12312016.csv")

In [21]:
#Informacion el dataset
inv_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224489 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InventoryId  224489 non-null  object 
 1   Store        224489 non-null  int64  
 2   City         223205 non-null  object 
 3   Brand        224489 non-null  int64  
 4   Description  224489 non-null  object 
 5   Size         224489 non-null  object 
 6   onHand       224489 non-null  int64  
 7   Price        224489 non-null  float64
 8   endDate      224489 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.4+ MB


In [22]:
#Primeras filas
inv_final.head()

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2016-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2016-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2016-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2016-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2016-12-31


In [23]:
#Estadisticas de cada columna
for i in inv_final.columns:
    print(i)
    #Esto imprime las estadísticas descriptivas (como count, mean, std, min, 25%, 50%, 75%, max) para esa columna.
    print(inv_final[i].describe())
    print('--------------------------------------------------------------')

InventoryId
count                224489
unique               224489
top       81_PEMBROKE_90604
freq                      1
Name: InventoryId, dtype: object
--------------------------------------------------------------
Store
count    224489.000000
mean         43.505740
std          23.326415
min           1.000000
25%          23.000000
50%          44.000000
75%          66.000000
max          81.000000
Name: Store, dtype: float64
--------------------------------------------------------------
City
count        223205
unique           67
top       MOUNTMEND
freq          18065
Name: City, dtype: object
--------------------------------------------------------------
Brand
count    224489.000000
mean      14356.370513
std       13118.467851
min          58.000000
25%        3798.000000
50%        8259.000000
75%       23965.000000
max       90631.000000
Name: Brand, dtype: float64
--------------------------------------------------------------
Description
count                   224489
u

In [24]:
# Convertimos la columna 'endDate' a tipo de dato datetime, usando el paramero errors="coerce"
inv_final['endDate'] = pd.to_datetime(inv_final['endDate'], errors='coerce')
print (inv_final['endDate'].dtypes)


datetime64[ns]


In [25]:
# Estadísticas Descriptivas - Variables Cuantitativas

inv_final.describe().T

,count,mean,min,25%,50%,75%,max,std
Store,224489.0,43.50574,1.0,23.0,44.0,66.0,81.0,23.326415
Brand,224489.0,14356.370513,58.0,3798.0,8259.0,23965.0,90631.0,13118.467851
onHand,224489.0,21.763988,0.0,7.0,12.0,22.0,3676.0,37.233576
Price,224489.0,23.585583,0.49,9.99,14.99,23.49,13999.9,79.202775
endDate,224489,2016-12-31 00:00:00,2016-12-31 00:00:00,2016-12-31 00:00:00,2016-12-31 00:00:00,2016-12-31 00:00:00,2016-12-31 00:00:00,NaN


In [26]:
#Exploración de Valores Anómalos
#onHand y Price verificar que estas columnas no tengan valores negativos, 
# ya que onHand debería ser positivo (o cero) y Price generalmente también.

inv_final = inv_final[inv_final['onHand'] >= 0]
inv_final = inv_final[inv_final['Price'] >= 0]

In [27]:
# Verificamos la cantidad de valores nulos por campo

print("Cantidad de valores nulos por columna:\n", inv_final.isnull().sum(), "\n")
print('------------------------')

Cantidad de valores nulos por columna:
 InventoryId       0
Store             0
City           1284
Brand             0
Description       0
Size              0
onHand            0
Price             0
endDate           0
dtype: int64 

------------------------


In [28]:
#Como en los demas archivos se encuestra la misma convercion de la columna Size, creando Size_ml
import re

def convertir_a_ml(size):
    # Expresión regular para extraer el número principal, unidad y posibles cantidades de paquetes
    match = re.match(r"(\d+(\.\d+)?)\s*(mL|L|liter|oz)?(?:\s*(\d+)?\s*Pk)?", str(size), re.IGNORECASE)
    
    if match:
        valor = float(match.group(1))  # Valor numérico principal
        unidad = match.group(3).lower() if match.group(3) else ''  # Unidad de medida
        cantidad_pk = int(match.group(4)) if match.group(4) else 1  # Cantidad en el paquete, por defecto 1
        
        # Convertir a mililitros según la unidad
        if unidad in ['l', 'liter']:
            return valor * 1000 * cantidad_pk  # Convertir litros a mililitros y multiplicar por cantidad
        elif unidad == 'oz':
            return valor * 29.5735 * cantidad_pk  # Convertir onzas a mililitros y multiplicar por cantidad
        elif unidad == 'ml':
            return valor * cantidad_pk  # Ya está en mililitros, multiplicar por cantidad
    return None

# Aplicar la función a la columna 'Size' para crear la columna 'Size_ml' en mL
inv_final['Size_ml'] = inv_final['Size'].apply(convertir_a_ml)

# Verificar el resultado
print(inv_final[['Size', 'Size_ml']].head(10))


         Size  Size_ml
0       750mL    750.0
1       750mL    750.0
2       750mL    750.0
3       750mL    750.0
4       750mL    750.0
5       750mL    750.0
6       750mL    750.0
7       Liter      NaN
8       Liter      NaN
9  750mL + 2/    750.0


In [29]:
# Eliminar filas donde 'Size_ml' tiene valores NaN
inv_final = inv_final.dropna(subset=['Size_ml'])

# Verificar el resultado
print(inv_final['Size_ml'].unique())


[  750.      1750.        50.       375.       200.       600.
   150.       400.       250.       100.       800.      1000.
  1500.      5000.      4000.       748.      3000.       147.8675
   561.       187.       500.      2250.       300.       330.
 18000.       720.      1125.       180.      6000.       374.
  4500.    ]


In [30]:
#Borramos la columna Size

del inv_final['Size']

In [31]:
inv_final.columns

Index(['InventoryId', 'Store', 'City', 'Brand', 'Description', 'onHand',
       'Price', 'endDate', 'Size_ml'],
      dtype='object')

In [32]:
# Encontramos que los nulos pertenecen a la misma tienda
# Reemplazar valores nulos en la columna 'City' con un valor específico

inv_final['City'].fillna('Not Specified', inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2068\3349267895.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  inv_final['City'].fillna('Not Specified', inplace = True)


In [33]:
# Verificar si aún quedan valores nulos

print("Valores nulos en 'City' después de reemplazo:", inv_final['City'].isnull().sum())

Valores nulos en 'City' después de reemplazo: 0


In [35]:
#Renombramos, Guardamos el dataframe en tipo csv y listo para normalizar.
inv_final = inv_final.rename(columns={
    'InventoryID': 'InventoryeID',
    'Store': 'StoreID',
    'City': 'City',
    'Brand': 'BrandID',
    'Description': 'Description',
    'onHand': 'OnHand',
    'Price': 'Price',
    'endDate': 'EndDate',
    'Size_ml': 'SizeMl'
    })
inv_final.info()
inv_final.to_csv (r"C:\Users\Admin\Desktop\proyecto final\Proceso\CSV listos\InventarioFinal.csv")

<class 'pandas.core.frame.DataFrame'>
Index: 220667 entries, 0 to 224488
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InventoryId  220667 non-null  object        
 1   StoreID      220667 non-null  int64         
 2   City         220667 non-null  object        
 3   BrandID      220667 non-null  int64         
 4   Description  220667 non-null  object        
 5   OnHand       220667 non-null  int64         
 6   Price        220667 non-null  float64       
 7   EndDate      220667 non-null  datetime64[ns]
 8   SizeMl       220667 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 16.8+ MB
